In [1]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [2]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [3]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients['SkinConditions'].head()

In [4]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [5]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [6]:
# Drop ID and location columns from dataframe
patientsTrimmed = patients.drop(['SFM Id', 'City', 'State', 'Country'], axis = 1)

In [7]:
# One-hot encode Gender column
patientsTrimmed['Gender'] = pd.Categorical(patientsTrimmed['Gender'].str.strip())
gender_onehot = pd.get_dummies(patientsTrimmed['Gender'], prefix = "Gender",
                                    prefix_sep = "-", dtype = int)
print(patientsTrimmed.head())
patientsTrimmed = patientsTrimmed.drop('Gender', axis = 1)
patientsTrimmed = patientsTrimmed.join(gender_onehot)
print(patientsTrimmed.head())

  Gender  BirthYear FitzpatrickSkinPhotoType SkinTone  \
0      M       2000                              dark   
1      F       1946                            medium   
2      F       1950                              fair   
3      M       1953                              fair   
4      F       1985                              fair   

                                      SkinConditions  
0                                               none  
1   sensitive-skin-allergist-diagnosed,sensitive-...  
2   sensitive-skin-allergist-diagnosed, eczema-at...  
3   sensitive-skin-allergist-diagnosed,skin-aller...  
4   sensitive-skin-allergist-diagnosed, sensitive...  
   BirthYear FitzpatrickSkinPhotoType SkinTone  \
0       2000                              dark   
1       1946                            medium   
2       1950                              fair   
3       1953                              fair   
4       1985                              fair   

                          

In [8]:
# One-hot encode SkinTone column
patientsTrimmed['SkinTone'] = pd.Categorical(patientsTrimmed['SkinTone'].str.strip())
skintone_onehot = pd.get_dummies(patientsTrimmed['SkinTone'], prefix = "SkinTone",
                                    prefix_sep = "-", dtype = int)
print(patientsTrimmed.head())
patientsTrimmed = patientsTrimmed.drop('SkinTone', axis = 1)
patientsTrimmed = patientsTrimmed.join(skintone_onehot)
print(patientsTrimmed.head())

   BirthYear FitzpatrickSkinPhotoType SkinTone  \
0       2000                              dark   
1       1946                            medium   
2       1950                              fair   
3       1953                              fair   
4       1985                              fair   

                                      SkinConditions  Gender-F  Gender-M  \
0                                               none         0         1   
1   sensitive-skin-allergist-diagnosed,sensitive-...         1         0   
2   sensitive-skin-allergist-diagnosed, eczema-at...         1         0   
3   sensitive-skin-allergist-diagnosed,skin-aller...         0         1   
4   sensitive-skin-allergist-diagnosed, sensitive...         1         0   

   Gender-Other  Gender-Undisclosed  
0             0                   0  
1             0                   0  
2             0                   0  
3             0                   0  
4             0                   0  
   BirthYear F

In [9]:
# One-hot encode FitzPatrickSkinPhotoType column
patientsTrimmed['FitzpatrickSkinPhotoType'] = pd.Categorical(patientsTrimmed['FitzpatrickSkinPhotoType'].str.strip())
# Dropping first here since it is a blank variable in the column
fitzpatrick_onehot = pd.get_dummies(patientsTrimmed['FitzpatrickSkinPhotoType'], prefix = "Fitzpatrick",
                                    prefix_sep = "-", drop_first = True, dtype = int)
print(patientsTrimmed.head())
patientsTrimmed = patientsTrimmed.drop('FitzpatrickSkinPhotoType', axis = 1)
patientsTrimmed = patientsTrimmed.join(fitzpatrick_onehot)
print(patientsTrimmed.head())

   BirthYear FitzpatrickSkinPhotoType  \
0       2000                            
1       1946                            
2       1950                            
3       1953                            
4       1985                            

                                      SkinConditions  Gender-F  Gender-M  \
0                                               none         0         1   
1   sensitive-skin-allergist-diagnosed,sensitive-...         1         0   
2   sensitive-skin-allergist-diagnosed, eczema-at...         1         0   
3   sensitive-skin-allergist-diagnosed,skin-aller...         0         1   
4   sensitive-skin-allergist-diagnosed, sensitive...         1         0   

   Gender-Other  Gender-Undisclosed  SkinTone-brown  SkinTone-dark  \
0             0                   0               0              1   
1             0                   0               0              0   
2             0                   0               0              0   
3             0 

In [10]:
# Switching to TextVectorization (Tokenizer is deprecated)
from keras.layers import TextVectorization
# Replace commas with whitespace
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].str.replace(',', ' ')
# Set the max length based on whitespace characters
max_len = patientsTrimmed['SkinConditions'].str.count(' ').max()
# Create TextVectorization object, separating on whitespace and using the max_len from earlier
vectorizer = TextVectorization(split = 'whitespace', output_sequence_length = max_len)

In [11]:
# Adapt using the column we want to convert
print(patientsTrimmed['SkinConditions'].head())
vectorizer.adapt(patientsTrimmed['SkinConditions'].values)
print(patientsTrimmed['SkinConditions'].head())
# Reset the column after converting values to vector and placing in array
skinConditions = vectorizer(patientsTrimmed['SkinConditions']).numpy()
patientsTrimmed = patientsTrimmed.drop('SkinConditions', axis = 1)

0                                                 none
1     sensitive-skin-allergist-diagnosed sensitive-...
2     sensitive-skin-allergist-diagnosed  eczema-at...
3     sensitive-skin-allergist-diagnosed skin-aller...
4     sensitive-skin-allergist-diagnosed  sensitive...
Name: SkinConditions, dtype: object

0                                                 none
1     sensitive-skin-allergist-diagnosed sensitive-...
2     sensitive-skin-allergist-diagnosed  eczema-at...
3     sensitive-skin-allergist-diagnosed skin-aller...
4     sensitive-skin-allergist-diagnosed  sensitive...
Name: SkinConditions, dtype: object


In [12]:
patientsArr = patientsTrimmed.values
input_data = np.concatenate((patientsArr, skinConditions), axis = 1)

In [13]:
# Drop ID column for preprocessing - ID should have no effect on prediction
allergiesNoId = allergies.drop('SFM Id', axis = 1)
# Remove all non-digit characters, then replace empty cells with NaN
allergiesNoId = allergiesNoId.replace(r'\D+', '', regex = True).replace('', np.nan)
# Set all NaN cells to 0
allergiesNoId = allergiesNoId.fillna(0)
# Convert entire dataframe to integer
allergiesNoId = allergiesNoId.astype(int)

In [14]:
allergiesNoId['AllergiesList'] = allergiesNoId.astype(str).apply(' '.join, axis=1)
allergiesNoId['AllergiesList'] = allergiesNoId['AllergiesList'].str.split()

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

allergiesArr = np.array(allergiesNoId['AllergiesList'])
# Multi-hot encode data
allergiesArray = mlb.fit_transform(allergiesArr)

In [16]:
allergiesNew = mlb.inverse_transform(allergiesArray)

In [17]:
allergiesDF = pd.DataFrame(mlb.transform(allergiesArr), columns = mlb.classes_)
allergiesDF.dtypes

0         int32
100612    int32
100613    int32
100702    int32
100857    int32
          ...  
9804      int32
98288     int32
99100     int32
9926      int32
99356     int32
Length: 731, dtype: object

In [29]:
from sklearn.model_selection import KFold
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from keras.layers import Dense
import keras
import numpy as np
    
cv = KFold(n_splits=5, shuffle=True, random_state=7869)

fold_count = 1
# Train the model for each split
# Define the model inside the for loop
for train, test in cv.split(input_data, allergiesArray):

    n_classes = 731

    # Input layer
    input_shape = (40,)
    inputs = keras.Input(input_shape)

    # Hidden layers
    x = Dense(256, activation='sigmoid')(inputs)
    x = Dense(512, activation='sigmoid')(x)

    # Output layer - use multilabel classification
    predictions = Dense(n_classes, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=predictions)
    model.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=0.35, gamma=14),
                  optimizer=keras.optimizers.Adam(learning_rate=0.1),
                  metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

    print("Running fold #: ", fold_count)

    fold_train_x = input_data[train]

    history = model.fit(
        fold_train_x, allergiesArray[train],
        epochs=20,
        verbose=1,
        validation_split=0.2
    )

    fold_test_x = input_data[test]
    y_true = allergiesArray[test]

    # Predict probabilities
    probs = model.predict(fold_test_x, verbose=1)

    # Dynamic threshold optimization based on F1 score
    best_f1 = 0
    optimal_threshold = 0
    for t in np.arange(0.1, 1, 0.025):
        y_pred = (probs > t).astype(int)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)
        if f1 > best_f1:
            best_f1 = f1
            optimal_threshold = t

    print("F1 score:", best_f1)
    print("Optimal threshold:", optimal_threshold)

    # Apply threshold
    y_pred = (probs > optimal_threshold).astype(int)

    # Compute precision and recall
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

    print("Precision:", precision)
    print("Recall:", recall)

    # Calculate the AUC
    auc = keras.metrics.AUC(multi_label=True, num_labels=731, from_logits=False)
    auc.update_state(y_true, probs)
    print("AUC:", auc.result().numpy())
    
    # Compute multilabel confusion matrix
    matrix = multilabel_confusion_matrix(y_true, y_pred)
    print(matrix)


    fold_count += 1

Running fold #:  1
Epoch 1/20
198/198 [==============================] - 2s 6ms/step - loss: 0.0389 - precision_18: 0.0816 - recall_18: 0.2325 - val_loss: 0.0326 - val_precision_18: 0.1365 - val_recall_18: 0.1535
Epoch 2/20
198/198 [==============================] - 1s 4ms/step - loss: 0.0117 - precision_18: 0.1671 - recall_18: 0.1424 - val_loss: 0.0109 - val_precision_18: 0.2014 - val_recall_18: 0.1416
Epoch 3/20
198/198 [==============================] - 1s 4ms/step - loss: 0.0073 - precision_18: 0.2294 - recall_18: 0.1344 - val_loss: 0.0092 - val_precision_18: 0.3413 - val_recall_18: 0.1440
Epoch 4/20
198/198 [==============================] - 1s 4ms/step - loss: 0.0063 - precision_18: 0.3934 - recall_18: 0.1336 - val_loss: 0.0074 - val_precision_18: 1.0000 - val_recall_18: 0.1406
Epoch 5/20
198/198 [==============================] - 1s 4ms/step - loss: 0.0055 - precision_18: 0.3927 - recall_18: 0.1334 - val_loss: 0.0067 - val_precision_18: 0.3338 - val_recall_18: 0.1408
Epoch 6/20


62/62 [==============================] - 0s 1ms/step
F1 score: 0.2496604810366574
Optimal threshold: 0.1
Precision: 0.22861177565124935
Recall: 0.9719387755102041
AUC: 0.3508892
[[[   0    0]
  [   0 1980]]

 [[1978    0]
  [   2    0]]

 [[   0 1977]
  [   0    3]]

 ...

 [[1979    0]
  [   1    0]]

 [[1979    0]
  [   1    0]]

 [[1979    0]
  [   1    0]]]
Running fold #:  3
Epoch 1/20
198/198 [==============================] - 2s 7ms/step - loss: 0.0273 - precision_20: 0.0753 - recall_20: 0.2276 - val_loss: 0.0175 - val_precision_20: 0.0852 - val_recall_20: 0.1638
Epoch 2/20
198/198 [==============================] - 1s 5ms/step - loss: 0.0086 - precision_20: 0.1804 - recall_20: 0.1382 - val_loss: 0.0099 - val_precision_20: 0.3346 - val_recall_20: 0.1378
Epoch 3/20
198/198 [==============================] - 1s 5ms/step - loss: 0.0065 - precision_20: 0.2780 - recall_20: 0.1337 - val_loss: 0.0082 - val_precision_20: 0.1251 - val_recall_20: 0.1373
Epoch 4/20
198/198 [===============

198/198 [==============================] - 1s 3ms/step - loss: 0.0078 - precision_21: 0.4374 - recall_21: 0.1331 - val_loss: 0.0103 - val_precision_21: 1.0000 - val_recall_21: 0.1389
Epoch 20/20
62/62 [==============================] - 0s 2ms/step
F1 score: 0.25085059629723544
Optimal threshold: 0.1
Precision: 0.22783215688989472
Recall: 0.974937343358396
AUC: 0.34336525
[[[   0    0]
  [   0 1979]]

 [[   0 1979]
  [   0    0]]

 [[   0 1977]
  [   0    2]]

 ...

 [[1976    0]
  [   3    0]]

 [[1979    0]
  [   0    0]]

 [[   0 1979]
  [   0    0]]]
Running fold #:  5
Epoch 1/20
198/198 [==============================] - 3s 4ms/step - loss: 0.0244 - precision_22: 0.0750 - recall_22: 0.2294 - val_loss: 0.0155 - val_precision_22: 0.0762 - val_recall_22: 0.1535
Epoch 2/20
198/198 [==============================] - 1s 3ms/step - loss: 0.0079 - precision_22: 0.1871 - recall_22: 0.1381 - val_loss: 0.0085 - val_precision_22: 0.2023 - val_recall_22: 0.1358
Epoch 3/20
198/198 [=============

In [19]:
# TODO: Save Keras model as separate file
model.save("model.h5")
vocab = vectorizer.get_vocabulary()
np.save("vocab.npy", vocab)

C:\Users\me\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
print(patients['FitzpatrickSkinPhotoType'].unique())
print(patients['Gender'].unique())
print(patients['SkinTone'].unique())

[' ' ' darker-white' ' white-fair' ' light-pale-white' ' brown'
 ' dark-brown-black' ' light-brown']
[' M' ' F' ' Undisclosed' ' Other']
[' dark' ' medium' ' fair' ' olive' ' light' ' brown']


In [21]:
print(input_data[0])

[2000    0    1    0    0    0    1    0    0    0    0    0    0    0
    0    0    0   14    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [22]:
import joblib
joblib.dump(mlb, "mlb.pkl")

['mlb.pkl']

In [23]:
mlb_new = joblib.load("mlb.pkl")
mlb_new.classes_

array(['0', '100612', '100613', '100702', '100857', '102', '10260',
       '103637', '104', '1043', '104630', '105017', '10537', '10538',
       '10539', '10541', '10544', '10546', '10552', '105611', '106344',
       '106518', '107036', '1107', '111858', '112', '112022', '1126',
       '113', '1130', '1131', '114559', '1149', '1153', '11661', '11662',
       '11686', '117', '1177', '11790', '119', '11996', '121146',
       '121641', '121642', '121652', '122055', '12344', '123717', '124',
       '124537', '124737', '12522', '128', '128564', '128565', '129742',
       '129747', '13', '130118', '13018', '130735', '131994', '131995',
       '131996', '131997', '132762', '133', '133255', '133260', '133266',
       '133268', '133269', '133270', '133271', '133272', '133273',
       '133274', '133275', '133628', '133629', '133714', '134', '134256',
       '134795', '134800', '135153', '136', '136640', '136703', '136704',
       '13691', '13708', '137787', '137788', '137789', '137790', '137791'